In [ ]:
%matplotlib inline
from numpy import *
from matplotlib.pyplot import *

# Weak Form - 1D

Given $V_h := \mathrm{span}\{\phi_i(x)\}_{i=0}^{n-1}$.
Find $u_h(x) \in V_h$ s.t.

$$
\int_{0}^1 u_h'\, \phi_i' = \int_0^1 f(x)\, \phi_i, \quad i = 0,\ldots, n-1
$$

$$
\int_{0}^1 u_h'\, \phi_i' = \left(\int_{0}^1 \phi_j'\, \phi_i'\right)\, u^j 
$$

This results in the following matrix:

$$
A_{ij} = \left(\int_{0}^1 \phi_j'\, \phi_i'\right)
$$

In order to perform this integration, we use some quadrature rules, chosen so that the product above is integrated exactly.

We start by defining the **Local Finite Element Space** of order $k$, defined using $\hat T := [0,1]$ (the **reference element**), $\hat{\mathcal{P}}^k([0,1])$ (the **local polynomial space**), and $\hat N$ (the **set of nodal functions**, that are a basis for the dual space of $\hat{\mathcal{P}}^k([0,1])$). 

We define everything in a very general way, so that we can extend it to high order polynomials (or two dimensions) in no time.

The basis functions, $V = \{v_i\}_{i=0}^{n-1}$, are defined implicitly, imposing orthogonality with $N := \{v^i\}_{i=0}^{n-1}$, i.e., $v^j(v_i) = \delta^j_i$.

In order for this to work, we need to first start from a known basis for $\mathcal{P}^k([0,1]) = \textrm{span}\{e_i\}_{i=0}^k =: \textrm{span} E$, and re-write our basis $V$ in terms of $E$.

$$
e_i = pow(x,i), \qquad v_i = v_i^k e_k \qquad v^j(v_i) = v^j(e_k) v^k_i 
$$

For each basis function $v_i$, we need to solve a system of equations in the $n$ unkonwns $v_i^k$ (free index $k$). Once this is done, we have our new polynomial basis functions (the **canonical basis** with respect to $N$):

$$
v^j(e_k) v^k_i  = C^j_k v^k_i = \delta^j_i \qquad i = 0, \ldots, n-1
$$

In [ ]:
from numpy.polynomial.polynomial import *

n = 5
# d = n-1

# Start by defining the local dual basis functions. These are linear functions on polynomials with
# values in R.
N = []

support_points = linspace(0,1,5)

for s in support_points:
    N.append(lambda f, s=s: f(s))

# Dimension of the polynomial space
n = len(N) 

# Construct the local space V, as the set of polynomials of order len(N)-1 such that the 
# dual basis above, evaluated on the basis, gives the identity matrix. We start by defining 
# monomials, so that we can use numpy Polynomial package
E = []
for i in range(len(N)):
    c = zeros((i+1,))
    c[-1] = 1
    E.append(Polynomial(c)) # Now E[i] contains the monomial x^i

# Construct the actual basis in terms of monomials,
# By solving for each basis a change of coordinate system
C = zeros((n,n)) # Matrix for the change of variables
for i in range(n):
    for j in range(n):
        C[i,j] = N[i](E[j])

# Auxiliary space to plot the result
s = linspace(0,1,1025)
V = []
for k in range(n):
    ei = zeros((n,))
    ei[k] = 1. # delta_ik
    vk = linalg.solve(C, ei)
    V.append(Polynomial(vk)) # Construct the basis using the right coefficients of the polynomials
    plot(s, V[k](s))

Now we have our local basis functions, constructed using **exclusively** the definition of the **Finite Element**, i.e., a reference element ([0,1]), a polynomial space, and a basis for the dual of the polynomial space.

In [ ]:
from numpy.polynomial.legendre import leggauss

nq = 2*len(N)-1
# degree = 10
q,w = leggauss(nq) # Gauss between -1 and 1
q = (q+1)/2 # Go back to 0,1 
w = w/2

print(q)

Now we evaluate all local basis functions and all derivatives of the basis functions at the quadrature points.

In [ ]:
# Lq_ijk = L[j]^(i)(q_k)
# ith derivative of jth basis function at point qk
n = len(N)
nq = len(q)
nder = 2
Lq = zeros((nder,n,nq))

for i in range(nder):
    for j in range(n):
        Lq[i][j] = V[j].deriv(i)(q)

$$
M_{ij} = \int_0^1 v_j(x) v_i(x) dx 
$$

$$
K_{ij} = \int_0^1 v'_j(x) v'_i(x) dx 
$$

$$
-\Delta u + u = f \quad \text{ in } \Omega
$$
$$
\frac{\partial u}{\partial n} = 0 \quad \text{ on } \partial \Omega
$$

## Weak form:
$$
\begin{split}
&(-\Delta u + u, v) = (f,v) \quad \forall v \in V \\
& (\nabla u, \nabla v) + (u,v) = (f,v) \quad \forall v \in V 
\end{split}
$$

$$
(u,v) := \int_\Omega u v dx
$$
 

$$
A_{ij} := (\nabla v_j, \nabla v_i) + (v_j, v_i)
$$

$$
(u,v) := \int_\Omega u v dx \sim \sum_{q=0}^{nq} u(x_q) v(x_q) w_q
$$

$$
M u = M_{ij} u^j = (v_j,v_i) u^j := \int_\Omega u^j v_j v_i dx \sim \sum_{q=0}^{nq} v_j(x_q) v_i(x_q) w_q u^j
$$

In [ ]:
M = einsum('jq, iq, q -> ij', Lq[0], Lq[0], w)
K = einsum('jq, iq, q -> ij', Lq[1], Lq[1], w)

A = M+K

# 2D

$$
\varphi_{ij}(x,y) := v_i(y) v_j(x), \qquad V:= \text{span}\{ \varphi_{ij} \}_{i,j=0}^{n,n}, \text{dim}(V) = n^2
$$

$$
\Omega = [0,1]^2, (u,v) := \int_0^1 \int_0^1 u_k(y) u_l(x) v_i(y) v_j(x) \, dx \, dy = \int_0^1 u_k(x) v_j(x) \, dx  \int_0^1 u_l(y) v_i(y)  \, dy 
$$

$$
u(x,y) := u^{kl} \varphi_{kl}(x,y)
$$
$$
v(x,y) := v^{ij} \varphi_{ij}(x,y)
$$

$$
M_{ijkl} := (\varphi_{kl}, \varphi_{ij})
$$

$$
K_{ijkl} := (\nabla \varphi_{kl}, \nabla \varphi_{ij})
$$

$$
M_{ijkl} := (\varphi_{kl}, \varphi_{ij}) = \int_0^1 \int_0^1 v_k(y) v_l(x) v_i(y) v_j(x) \, dx \, dy \sim \sum_{a=0}^{nq-1} \sum_{b=0}^{nq-1} v_k(y_a) v_l(x_b) v_i(y_a) v_j(x_b) w_a w_b
$$


$$
\nabla \varphi_{ij} := [ \partial_x \varphi_{ij}(x,y)  , \partial_y  \varphi_{ij} (x,y) ] = [ v_i(y) v'_j(x), v'_i(y) v_j(x) ]
$$

$$
\nabla \varphi_{kl} := [ \partial_x \varphi_{kl}(x,y)  , \partial_y  \varphi_{kl} (x,y) ] = [ v_k(y) v'_l(x), v'_k(y) v_l(x) ]
$$
$$
\nabla \varphi_{ij} \cdot \nabla \varphi_{kl}  :=  v_i(y) v'_j(x) v_k(y) v'_l(x) +  v'_i(y) v_j(x) v'_k(y) v_l(x)
$$


in 3D:

$$
\nabla \varphi_{ijk} \cdot \nabla \varphi_{lmn}  :=  
v_i(z) v_j(y) v'_k(x)   v_l(z) v_m(y) v'_n(x) +
v_i(z) v_j'(y) v_k(x)   v_l(z) v'_m(y) v_n(x) +
v'_i(z) v_j(y) v_k(x)   v'_l(z) v_m(y) v_n(x)
$$



In [ ]:
M2d =  einsum('ia, jb, ka, lb, a, b -> ijkl', Lq[0], Lq[0], Lq[0], Lq[0], w, w)
K2d =  einsum('ia, jb, ka, lb, a, b -> ijkl', Lq[0], Lq[1], Lq[0], Lq[1], w, w)
K2d += einsum('ia, jb, ka, lb, a, b -> ijkl', Lq[1], Lq[0], Lq[1], Lq[0], w, w)

A2d = M2d+K2d

In [ ]:
AA = A2d.reshape((n*n, n*n))
AA.shape

In [ ]:
u = -cos(2*pi*q)
up = 2*pi*sin(2*pi*q)
upp = 4*pi**2*cos(2*pi*q)

f = -upp + u 

plot(q,u)

# Manufactured solution in 2D

$$
u(x) = -cos(2\pi x)
$$

$$
U(x,y) = u(x)u(y)
$$

$$
-\Delta U = - \partial_{xx} U - \partial_{yy} U = -u''(x)u(y) - u''(y)u(x)
$$

$$
F = -\Delta U + U = -u''(x)u(y) - u''(y)u(x) + u(x)u(y)
$$

$$
F_{ab} := -u(y_a) u''(x_b) - u''(y_a)u(x_b) + u(y_a)u(x_b)
$$

Rhs of Finite Element Method: 

$$
RHS_{ij} = (F,\varphi_{ij}) := \sum^{nq-1}_{a=0}\sum^{nq-1}_{b=0} F_{ab} v_i(y_a) v_j(x_b) w_a w_b
$$


In [ ]:
k = 2*pi
u = cos(k*q)
up = -k*sin(k*q)
upp = -k**2*cos(k*q)

f = -upp + u 

F = einsum('ia, jb, a, b, a, b', Lq[0], Lq[0], u, -upp, w, w)
F += einsum('ia, jb, a, b, a, b', Lq[0], Lq[0], -upp, u, w, w)
F += einsum('ia, jb, a, b, a, b', Lq[0], Lq[0], u, u, w, w)

b = F.reshape((n**2, 1))

In [ ]:
# solve: AA.dot(x) = b

x = np.linalg.solve(AA,b) 

In [ ]:
# Plots
ns = 51
s = linspace(0,1,ns)

Ls = zeros((n, ns))
for j in range(n):
    Ls[j] = V[j](s)

In [ ]:
# 2D Interp matrix:

C = einsum('is, jk -> skij', Ls, Ls)

Xs = einsum('skij, ij', C, x.reshape(n,n))
X, Y = meshgrid(s,s)


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(X,Y,Xs)

See 

https://www.dealii.org/8.4.0/doxygen/deal.II/step_37.html

for an explanation of how to do "matrix free" stuff faster!

# Explanation of Matrix free.

$$
M_{ij} = (v^t v)_{ij} = v_i v_j
$$

Size of $M = n \times n$. Cost of $ M u$ is $O(n^2)$.


If you do $ v_j \cdot u_j$, and then *scale* the result with $v_i$, the cost is $O(n)$.

$$
M_{ij} u^j = \sum_{j=0}^{n-1} M_{ij} u^j,  i = 0 \ldots n-1
$$

By definition of $M$, though:

$$
M_{ij} = v_i v_j
$$

then 

$$
M_{ij} u^j := \big(\sum_{j=0}^{n-1} v_j u^j\big) v_i
$$

Call $ a := \sum_{j=0}^{n-1} v_j u^j$ (cost $O(n)$). 

$$
M_{ij} u^j = a v_i 
$$

cost of $a v_i$ is $O(n)$


